In [44]:
#import libraries
import pandas
import pandas as pd
import numpy
from numpy import arange
#imported for data scaling
from sklearn.preprocessing import MinMaxScaler
#imported to split data into training and testing sets for each model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold 
#import 3 models to compare accuracy
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LassoCV
#imported two metrics used to determine accuracy of models
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [45]:
#imports streamflow data from the 9 subbasins draining to the Mississippi-Atchafalaya basin, which drains into Gulf of Mexico (input)
canneltondata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/canneltonmonthly.csv')
canf = pd.DataFrame(canneltondata)

clintondata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/clintonmonthly.csv')
clintonf = pd.DataFrame(clintondata)

graftondata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/graftonmonthly.csv')
graftonf = pd.DataFrame(graftondata)

grandchaindata = pd.read_csv('/Users/Taj//Desktop/github/Predicting-Nitrogen-Contamination/CleanData/grandchainmonthly.csv')
grandf = pd.DataFrame(grandchaindata)

hermandata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/hermanmonthly.csv')
hermanf = pd.DataFrame(hermandata)

littlerockdata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/littlerockmonthly.csv')
littlef = pd.DataFrame(littlerockdata)

omahadata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/omahamonthly.csv')
omahaf = pd.DataFrame(omahadata)

redriverdata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/redrockmonthly.csv')
redf = pd.DataFrame(redriverdata)

thebesdata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/thebesmonthly.csv')
thebesf = pd.DataFrame(thebesdata)

#imports monthly precipitation data in the Midwest (input)
precipdata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/precipmonthly.csv')
precipf = pd.DataFrame(precipdata)

#imports data of nitrogen concentration in the Gulf of Mexico monthly (output)
outputdata = pd.read_csv('/Users/Taj/Desktop/github/Predicting-Nitrogen-Contamination/CleanData/gulfmonthly.csv')
outputdf = pd.DataFrame(outputdata)

## Cleaning Data

### Nitrogen Contamination in Gulf of Mexico (Output Data)

In [46]:
#drops NaN (blank) values
outputdf = outputdf.dropna()
#renames columns to be more readable
outputdf.columns = ['Months', 'Conc']
#resets index to start from 0
outputdf.reset_index(drop=True, inplace=True)
outputdf.head()

,Months,Conc
0,10/1/1978,22600.0
1,11/1/1978,18500.0
2,12/1/1978,65300.0
3,1/1/1979,75700.0
4,2/1/1979,68600.0


### Subbasin Streamflow (Input Data)

In [47]:
#defines function to clean streamflow data, used by the 9 subbasins
def cleansflow(name, dfname):
    #renames columns to be more readable and standardized for each dataset
    dfname.columns = ['Months', name + ' Streamflow']
    #drops NaN (blank) values
    dfname = dfname.dropna()

In [48]:
#calls function to clean streamflow data for each of the 9 subbasins
cleansflow('Clinton', clintonf)
cleansflow('Grafton', graftonf)
cleansflow('Herman', hermanf)
cleansflow('Omaha', omahaf)
cleansflow('Red River', redf)
cleansflow('Cannelton', canf)
cleansflow('Grand Chain', grandf)
cleansflow('Little Rock', littlef)
cleansflow('Thebes', thebesf)

### Monthly Midwest Precipitation (Input Data)

In [49]:
#renames Months column to be standardized with other dataframes
precipf.columns = ['Months', 'Precip']

## Merge All DataFrames

In [50]:
#lists all dataframes which need to be merged
merged = [outputdf, clintonf, graftonf, hermanf, omahaf, redf, canf, grandf, littlef, thebesf, precipf]

#creates final dataframe with all input and output
alldf = outputdf

#merges two dataframes together into alldf then merges alldf with the next dataframe and so on
for i in range(0, 10):
    alldf = pd.merge(alldf, merged[i+1], how='inner', on='Months')
    
#drops rows with NaN (blank) values
alldf.dropna(inplace = True)

alldf.head()

,Months,Conc,Clinton Streamflow,Grafton Streamflow,Herman Streamflow,Omaha Streamflow,Red River Streamflow,Cannelton Streamflow,Grand Chain Streamflow,Little Rock Streamflow,Thebes Streamflow,Precip
0,10/1/1978,22600.0,1080.0,1900.0,1920.0,1610.0,91.7,1250.0,2570.0,41.5,4100.0,1.89
1,11/1/1978,18500.0,910.0,1600.0,2200.0,1610.0,272.0,1840.0,3630.0,236.0,3940.0,2.87
2,12/1/1978,65300.0,593.0,1470.0,1530.0,895.0,412.0,9460.0,18500.0,422.0,3290.0,2.86
3,1/1/1979,75700.0,502.0,1140.0,917.0,619.0,1460.0,9210.0,20900.0,670.0,2330.0,2.57
4,2/1/1979,68600.0,546.0,1520.0,1910.0,710.0,1530.0,5340.0,11700.0,904.0,3780.0,1.77


# Fitting Regression Models

In [51]:
#sets independent values (x) to all features (subbasin and precipitation data)
x = alldf.drop(['Months', 'Conc'], axis = 1)

#sets dependent value (y) to response (concentration of nitrogen in the Gulf of Mexico)
y = alldf['Conc']

#convert y from a 1d array to a 2d array
yarray = numpy.array(y.values.tolist())
y = yarray.reshape((444, 1))

#scale x values (independent variables) to account for different units
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
y = scaler.fit_transform(y)

In [52]:
#split data into training and testing sets for the models
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.5, random_state = 42)

## Ridge Regression

In [53]:
#define model (tests which alpha to use using k-fold cv)
cv = RepeatedKFold(n_splits=5, n_repeats=4, random_state=1)
ridge = RidgeCV(alphas=arange(0, 2, 0.01), cv = cv)

#fit data to model
ridge.fit(x_train, y_train)

#predict y values on test set
y_predRidge = ridge.predict(x_test)

#evaluate model using MSE and R2 score
mseR = mean_squared_error(y_test, y_predRidge)
r2R = r2_score(y_test, y_predRidge)
print("Alpha: " + str(ridge.alpha_))
print("MSE: " + str(mseR))
print("R2 score: " + str(r2R))
print("Coeffecients:" + str(ridge.coef_))

Alpha: 0.44
MSE: 0.008639321655160693
R2 score: 0.8315643303001811
Coeffecients:[[ 0.16325898  0.34643915  0.14429136  0.02116497  0.18802257  0.0630726
   0.24450559  0.07965137  0.28746976 -0.12191444]]


## LASSO Regression

In [54]:
#define model (tests which alpha to use using k-fold cv)
lasso = LassoCV(alphas=None, cv = cv)

#changes y to a 1d array
y_train = y_train.ravel()

#fit data to model
lasso.fit(x_train, y_train)

#predict y values on test set
y_predLasso = lasso.predict(x_test)

#evaluate model using MSE and R2 score
mseL = mean_squared_error(y_test, y_predLasso)
r2L = r2_score(y_test, y_predLasso)
print("Alpha: " + str(lasso.alpha_))
print("MSE: " + str(mseL))
print("R2 score: " + str(r2L))
print("Coeffecients:" + str(lasso.coef_))


Alpha: 0.0002699950530389419
MSE: 0.00840298154563718
R2 score: 0.8361721115836506
Coeffecients:[ 0.13048391  0.45019242  0.13297976  0.          0.18744616  0.
  0.30634142  0.06911331  0.24228461 -0.12411852]


## ElasticNet Regression

In [55]:
#define model (tests which alpha to use using k-fold cv)
elastic = ElasticNetCV(alphas=None, cv = cv)

#fit data to model
elastic.fit(x_train, y_train)

#predict y values on test set
y_predElastic = elastic.predict(x_test)

#evaluate model using MSE and R2 score
mseE = mean_squared_error(y_test, y_predElastic)
r2E = r2_score(y_test, y_predElastic)
print("Alpha:" + str(elastic.alpha_))
print("MSE: " + str(mseE))
print("R2 score: " + str(r2E))
print("Coeffecients:" + str(elastic.coef_))

Alpha:0.000539990106077884
MSE: 0.008375982614254372
R2 score: 0.8366984935462813
Coeffecients:[ 0.13906891  0.40280354  0.11631166  0.          0.18767353  0.00503105
  0.29832503  0.06870463  0.28847722 -0.12083567]
